In [51]:
import numpy as np
import pandas as pd
import os
import sys
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
# cleanse_pipe is a cleanse script I made for this project.
from cleanse_pipe import cleanse_data
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import math
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import time
import pycaret
from sklearn.svm import LinearSVC

from pycaret.classification import *

# This is my next stab at improving my model
from sklearn.ensemble import VotingClassifier

In [52]:
train = pd.read_csv('train.csv')
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [53]:
data, passengers = cleanse_data(train)

del[data['Age'], data['Fare'], data['Embarked_C']]
# %%
data.head()

,Survived,Pclass,SibSp,Parch,Embarked_Q,Embarked_S,Alone,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Ms,Sex_Male,Farebin,Agebin
0,0,3,1,0,0,1,0,0,0,1,0,0,1,2,2
1,1,1,1,0,0,0,0,0,0,0,1,0,0,5,4
2,1,3,0,0,0,1,1,0,1,0,0,0,0,2,2
3,1,1,1,0,0,1,0,0,0,0,1,0,0,5,4
4,0,3,0,0,0,1,1,0,0,1,0,0,1,2,4


In [54]:
data['Survived'].mean()

0.3838383838383838

In [55]:
# normalize data since we are going to be using models that need normalization
exp_clf = setup(data, target = 'Survived', train_size = .85, normalize= False, log_experiment= False)

,Description,Value
0,session_id,6879
1,Target,Survived
2,Target Type,Binary
3,Label Encoded,"0: 0, 1: 1"
4,Original Data,"(891, 15)"
5,Missing Values,False
6,Numeric Features,7
7,Categorical Features,7
8,Ordinal Features,False
9,High Cardinality Features,False


In [56]:
# exp_clf[0].head()

#%%

best = compare_models()
# %%


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.8441,0.8794,0.7275,0.8400,0.7743,0.6573,0.6656,0.0170
lr,Logistic Regression,0.8361,0.8686,0.7346,0.8162,0.7703,0.6439,0.6488,0.0080
lightgbm,Light Gradient Boosting Machine,0.8335,0.8619,0.7100,0.8258,0.7592,0.6340,0.6416,0.0110
ridge,Ridge Classifier,0.8308,0.0000,0.7379,0.8009,0.7656,0.6339,0.6375,0.0060
lda,Linear Discriminant Analysis,0.8295,0.8681,0.7344,0.8002,0.7632,0.6307,0.6345,0.0060
xgboost,Extreme Gradient Boosting,0.8282,0.8549,0.7099,0.8172,0.7553,0.6245,0.6320,0.0890
catboost,CatBoost Classifier,0.8282,0.8756,0.7101,0.8142,0.7547,0.6242,0.6309,1.0150
rf,Random Forest Classifier,0.8269,0.8547,0.7099,0.8109,0.7529,0.6213,0.6279,0.0610
et,Extra Trees Classifier,0.8269,0.8278,0.6995,0.8192,0.7507,0.6201,0.6277,0.0430
knn,K Neighbors Classifier,0.8203,0.8466,0.6821,0.8182,0.7399,0.6049,0.6141,0.0120


In [57]:
catboost = create_model('gbc')


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7500,0.8162,0.4828,0.7778,0.5957,0.4288,0.4544
1,0.8553,0.9032,0.8621,0.7812,0.8197,0.6993,0.7016
2,0.8684,0.9497,0.8621,0.8065,0.8333,0.7248,0.7259
3,0.8684,0.9193,0.7586,0.8800,0.8148,0.7136,0.7184
4,0.8026,0.8536,0.6552,0.7917,0.7170,0.5675,0.5735
5,0.9211,0.9365,0.8966,0.8966,0.8966,0.8327,0.8327
6,0.8816,0.9292,0.7931,0.8846,0.8364,0.7440,0.7467
7,0.8133,0.8385,0.6429,0.8182,0.7200,0.5830,0.5925
8,0.8400,0.7751,0.6786,0.8636,0.7600,0.6426,0.6531
9,0.8400,0.8723,0.6429,0.9000,0.7500,0.6371,0.6566


In [58]:
tuned_cat = tune_model(catboost, n_iter= 100)



,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7763,0.8313,0.5517,0.8000,0.6531,0.4961,0.5148
1,0.8684,0.9028,0.8621,0.8065,0.8333,0.7248,0.7259
2,0.8553,0.9413,0.8276,0.8000,0.8136,0.6953,0.6956
3,0.8684,0.8933,0.7586,0.8800,0.8148,0.7136,0.7184
4,0.8158,0.8258,0.6897,0.8000,0.7407,0.5991,0.6031
5,0.9079,0.9266,0.8276,0.9231,0.8727,0.8009,0.8038
6,0.8816,0.9281,0.7586,0.9167,0.8302,0.7405,0.7483
7,0.7867,0.8381,0.6071,0.7727,0.6800,0.5234,0.5320
8,0.8400,0.7500,0.7143,0.8333,0.7692,0.6479,0.6524
9,0.8400,0.8628,0.6786,0.8636,0.7600,0.6426,0.6531


In [59]:
tuned_cat

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=11,
                           max_features=1.0, max_leaf_nodes=None,
                           min_impurity_decrease=0.4, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=10,
                           min_weight_fraction_leaf=0.0, n_estimators=70,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=6879, subsample=0.95, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [60]:
test = pd.read_csv('test.csv')
test_data, passengers1 = cleanse_data(test)

del[test_data['Age'], test_data['Fare'], test_data['Embarked_C']]

preds = predict_model(tuned_cat, data = test_data)

ready_preds = pd.DataFrame({"PassengerId" : passengers1, "Survived" : preds['Label']})

ready_preds.to_csv('titanic_preds.csv', index = False)

In [61]:
ready_preds.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [62]:
ready_preds['Survived'].mean()

0.36363636363636365

In [63]:
data1 = data.copy()
target = data1['Survived']
del[data1['Survived']]

# del[data1['Age'], data1['Fare'], data1['Embarked_C']]

data1.head()

,Pclass,SibSp,Parch,Embarked_Q,Embarked_S,Alone,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Ms,Sex_Male,Farebin,Agebin
0,3,1,0,0,1,0,0,0,1,0,0,1,2,2
1,1,1,0,0,0,0,0,0,0,1,0,0,5,4
2,3,0,0,0,1,1,0,1,0,0,0,0,2,2
3,1,1,0,0,1,0,0,0,0,1,0,0,5,4
4,3,0,0,0,1,1,0,0,1,0,0,1,2,4


In [64]:
data1.describe()

,Pclass,SibSp,Parch,Embarked_Q,Embarked_S,Alone,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Ms,Sex_Male,Farebin,Agebin
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,2.308642,0.523008,0.381594,0.086420,0.725028,0.602694,0.056117,0.204265,0.592593,0.140292,0.006734,0.647587,3.204265,2.940516
std,0.836071,1.102743,0.806057,0.281141,0.446751,0.489615,0.230276,0.403390,0.491628,0.347485,0.081830,0.477990,1.208037,1.362354
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
25%,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,2.000000
50%,3.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,3.000000,3.000000
75%,3.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,4.000000,4.000000
max,3.000000,8.000000,6.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,5.000000,5.000000


In [65]:
# Standardize continuous variables

def standardScale_custom(x) :
    new_data = []
    stan_mean = np.mean(x)
    stan_std = np.std(x)
    for row in x :
        new_data.append((row - stan_mean) / stan_std)
    return new_data
    
#for col in ['Age','Fare'] :
 #   data1[col] = standardScale_custom(data1[col])


In [66]:
x_train, x_test, y_train, y_test = train_test_split(data1, target, test_size=0.15, random_state= 546)

In [67]:
print(y_train.mean(), y_test.mean())

0.3817701453104359 0.39552238805970147


In [68]:
svm = LinearSVC(penalty = 'l2', loss = 'squared_hinge', C = 0.5, fit_intercept= True, max_iter = 1000)

svm_fit = svm.fit(x_train, y_train)
svm_predict = svm_fit.predict(x_train)

print(classification_report(y_train, svm_predict))

              precision    recall  f1-score   support

           0       0.85      0.89      0.87       468
           1       0.81      0.75      0.78       289

    accuracy                           0.84       757
   macro avg       0.83      0.82      0.83       757
weighted avg       0.84      0.84      0.84       757



In [69]:
search_params = {"C" : [.01,.5,.8,1,2,5,7,10],
    "loss" : ['hinge', 'squared_hinge'],
    "fit_intercept" : [True,False]
}

grid = GridSearchCV(svm, param_grid= search_params, n_jobs = -1, cv = 10)

grid_fit = grid.fit(x_train, y_train)

print(grid_fit.best_params_)
print(grid_fit.best_score_)



{'C': 0.8, 'fit_intercept': True, 'loss': 'squared_hinge'}
0.8375789473684211


In [70]:
test = pd.read_csv('test.csv')
test_data, passengers1 = cleanse_data(test)

del[test_data['Age'], test_data['Fare'], test_data['Embarked_C']]

preds = grid_fit.predict(test_data)

read_preds = pd.DataFrame({"PassengerId" : passengers1, "Survived" : preds})

ready_preds.to_csv('titanic_preds.csv', index = False)

In [71]:
read_preds.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [72]:
read_preds['Survived'].mean()

0.3875598086124402

In [75]:
test = pd.read_csv('test.csv')
test_data, passengers1 = cleanse_data(test)

#for col in ['Age','Fare'] :
 #   test_data[col] = standardScale_custom(test_data[col])

del[test_data['Age'], test_data['Fare'], test_data['Embarked_C']]


test_predict = voting_fit.predict(test_data)

ready_preds = pd.DataFrame({'PassengerId' : passengers1, "Survived" : test_predict})

ready_preds.to_csv('titanic_preds.csv', index = False)